In [7]:
from owlready2 import *
from owlready2.pymedtermino2 import *
from owlready2.pymedtermino2.umls import *
# from owlready2.pymedtermino2.icd10_french import *

from tqdm import tqdm
import pandas as pd

from collections import defaultdict
import random
import networkx as nx
from tqdm import tqdm

import random

In [40]:
DB_NAME = "../../../data/pym.sqlite3"
ZIP_NAME = "../../../data/umls-2024AA-full.zip"
SAVE_TO = "../../../data/seqs.tsv"


# previously deleted:     'term_type', 'terminology', 
# added: 'ctv3id', originals'
BANNED_PROPS = [
    'icd-o-3_code',  #ok almost empty
    'ctv3id', #ok unique for all, which is bad, full memorization
    'subset_member',  #ok unique for all, which is bad, full memorization
    'label', #ok almost the same
    'synonyms',  #ok almost the same to the parent node
    'unifieds', #ok almost the same to the parent node
    'originals'  #ok almost the same to the parent node
               ]
RANDOM_SEED=30_239_566

BANNED_GROUP = ['case_significance_id',
 'groups',
 'type_id',
 'ctv3id',
 'effective_time',
 'unifieds',
 'active',
 'synonyms',
 'terminology',
 'subset_member',
 'definition_status_id',
 'term_type']

# mapped_to: example -- 'ICD10["I51.8"] # Other ill-defined heart diseases\n'


In [41]:
default_world.set_backend(filename = DB_NAME)
PYM = get_ontology("http://PYM/").load()

In [42]:
# Graph initialization from SNOMED data
def build_graph(ontology):
    G = nx.Graph()
    for concept in tqdm(ontology.classes()):
        for prop in concept.get_class_properties():
            if prop.name in BANNED_PROPS or prop.name in BANNED_GROUP:
                continue
            related_concepts = getattr(concept, prop.name, [])
            if not isinstance(related_concepts, (list, set)):
                related_concepts = [related_concepts]
            for rc in related_concepts:
                G.add_edge(concept, rc)
    return G

In [43]:
ontology = PYM
G = build_graph(ontology)

1220039it [02:14, 9080.57it/s] 


In [54]:
# import networkx as nx
# import random
# from graphviz import Digraph

# # Ensure G is defined, and iterate until a suitable subgraph is found
# subgraph_nodes = []
# while len(subgraph_nodes) < 4:  # Ensure there are at least 2 nodes in the subgraph
#     start_node = random.choice(list(G.nodes))
#     bfs_nodes = nx.single_source_shortest_path_length(G, start_node, cutoff=2)
#     subgraph_nodes = list(bfs_nodes.keys())

# # Create a subgraph with the selected nodes
# subgraph = G.subgraph(subgraph_nodes)

# # Initialize a Digraph for visualization
# dot = Digraph(format="png")
# dot.attr(rankdir="LR", size="10,5")

# # Add nodes and edges to the Digraph
# for node in subgraph.nodes:
#     dot.node(str(node))

# for source, target in subgraph.edges:
#     dot.edge(str(source), str(target))

# Save and render the graph
output_path = "../../../data/random_graph"
dot.render(output_path, cleanup=True)

output_path + ".png"

 unrecognized


'../../../data/random_graph.png'

In [57]:
nodes_text = f"Nodes in subgraph: {list(subgraph.nodes)}\n"
edges_text = f"Edges in subgraph: {list(subgraph.edges)}"
nodes_text, edges_text

('Nodes in subgraph: [groups.3866206_3, groups.3815811_1, groups.3863669_1, SNOMEDCT_US["768843007"] # DNMT3A-related overgrowth syndrome\n, groups.3728927_1, SNOMEDCT_US["735749005"] # Myelomeningocele and hydrocephalus\n, groups.3845431_2, SNOMEDCT_US["838300000"] # Blue sclera of bilateral eyes\n, groups.3805258_2, groups.3532930_1, groups.3591268_1, groups.3592086_2, groups.3815811_2, groups.3802319_2, groups.3698401_1, groups.3479779_1, groups.3764780_4, groups.3815084_1, groups.3479987_1, SNOMEDCT_US["840426005"] # Bilateral developmental anomaly of pleurae\n, SNOMEDCT_US["766824003"] # ADNP-related multiple congenital anomalies, intellectual disability, autism spectrum disorder\n, groups.3748590_1, groups.3646770_1, SNOMEDCT_US["773547003"] # 13q12.3 microdeletion syndrome\n, groups.3901758_1, groups.3842765_1, SNOMEDCT_US["95198001"] # Pure gonadal dysgenesis 46,XX\n, groups.3866009_2, SNOMEDCT_US["840495000"] # Common atrioventricular junction with spontaneous fibrous closure 

In [45]:
! pip install graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 1.4 MB/s eta 0:00:00


In [13]:
G.degree

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/I6356345/anaconda3/envs/p37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_218462/967527955.py", line 1, in <module>
    G.degree
  File "/home/I6356345/anaconda3/envs/p37/lib/python3.7/site-packages/IPython/core/displayhook.py", line 262, in __call__
    format_dict, md_dict = self.compute_format_data(result)
  File "/home/I6356345/anaconda3/envs/p37/lib/python3.7/site-packages/IPython/core/displayhook.py", line 151, in compute_format_data
    return self.shell.display_formatter.format(result)
  File "/home/I6356345/anaconda3/envs/p37/lib/python3.7/site-packages/IPython/core/formatters.py", line 180, in format
    data = formatter(obj)
  File "/home/I6356345/anaconda3/envs/p37/lib/python3.7/site-packages/decorator.py", line 232, in fun
    return caller(func, *(extras + args), **kw)
  File "/home/I6356345/anaconda3/envs/p3

TypeError: object of type 'NoneType' has no len()

# havent checked code below

In [ ]:
# Sequence generation functions
def generate_condensed_sequences(G, num_samples, length_range, seed=None):
    random.seed(seed)
    sequences = []
    clusters = nx.algorithms.community.greedy_modularity_communities(G)
    for _ in tqdm(range(num_samples)):
        cluster = random.choice(clusters)
        start_node = random.choice(list(cluster))
        sequence = [start_node]
        while len(sequence) < random.randint(*length_range):
            neighbors = list(G.neighbors(sequence[-1]))
            neighbors = [n for n in neighbors if n in cluster]
            if not neighbors:
                break
            sequence.append(random.choice(neighbors))
        if len(sequence) >= length_range[0]:
            sequences.append(sequence)
    return sequences

def generate_sparse_sequences(G, num_samples, length_range, seed=None):
    random.seed(seed)
    sequences = []
    for _ in tqdm(range(num_samples)):
        start_node = random.choice(list(G.nodes))
        sequence = [start_node]
        while len(sequence) < random.randint(*length_range):
            neighbors = list(G.neighbors(sequence[-1]))
            neighbors = [n for n in neighbors if n not in sequence]  # Avoid dense local regions
            if not neighbors:
                break
            sequence.append(random.choice(neighbors))
        if len(sequence) >= length_range[0]:
            sequences.append(sequence)
    return sequences

# Convert sequences to a human-readable format
def format_sequence(sequence):
    return " -> ".join([str(node) for node in sequence])

# Main workflow
def create_datasets(ontology, num_samples=20000, length_range=(4, 7), seed=42):
    G = build_graph(ontology)
    condensed = generate_condensed_sequences(G, num_samples, length_range, seed)
    sparse = generate_sparse_sequences(G, num_samples, length_range, seed)
    # Save datasets to files
    with open("condensed_dataset.txt", "w") as f:
        for seq in condensed:
            f.write(format_sequence(seq) + "\n")
    with open("sparse_dataset.txt", "w") as f:
        for seq in sparse:
            f.write(format_sequence(seq) + "\n")

# Call the function
create_datasets(PYM, num_samples=20000, length_range=(4, 7), seed=RANDOM_SEED)

In [4]:
# def get_concept_triplets(concept, seed=None):
#     triplets = []
    
#     # Set random seed for reproducibility if provided
#     if seed is not None:
#         random.seed(seed)
    
#     # Iterate through all properties of the concept
#     for prop in concept.get_class_properties():
#         # Skip the banned properties
#         if prop.name in BANNED_PROPS: 
#             continue
        
#         # Get the linked concept(s) via the property
#         related_concepts = getattr(concept, prop.name)
        
#         selected_concept = related_concepts
        
#         # If related concepts are a list or set, select one randomly (not single concept)
#         if (isinstance(related_concepts, list) or isinstance(related_concepts, set)) and related_concepts:
#             selected_concept = random.choice(list(related_concepts))
            
#         a,b,c = str(concept.label[0]), prop.name, str(selected_concept)
#         if hasattr(selected_concept, 'label') and selected_concept.label:
#             a,b,c = str(concept.label[0]), prop.name, str(selected_concept.label[0])
# #         if 'id' not in b: #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# #             print('KEK', b)
# #             print(a,b,c)
# #             continue
# #         if b in ID_NAMES:
# #             print('LOL', b)
# #             continue
# #         else:
# #             ID_NAMES.append(b)

#         triplets.append((a,b,c))
#         print(a,b,c, sep = ' | ')
    
#     return triplets

# # Function to get triplets for all concepts in the ontology
# def get_all_concept_triplets(ontology, seed=None):
#     all_triplets = []

#     # Iterate over all concepts (classes) in the ontology
# #     i=0
#     for concept in tqdm(ontology.classes()):
# #         i+=1
# #         if i > 100000:
# #             break
#         try:
#             triplets = get_concept_triplets(concept, seed)  # Use the function from before
#             all_triplets.extend(triplets)  # Add the triplets to the full list
#         except IndexError:
#             pass
#     return all_triplets

In [ ]:
all_triplets = get_all_concept_triplets(PYM, seed=RANDOM_SEED)

371343it [01:19, 2597.24it/s]

Aeroallergen | module_id | 900000000000012004


371752it [01:20, 1373.49it/s]

Allergen | module_id | 900000000000012004
Alternaria | module_id | 731000124108
Amaranth | module_id | 731000124108
Ambulatory care site | module_id | 731000124108
Anemia of chronic disease | module_id | 731000124108


372550it [01:21, 1496.44it/s]

...

651817it [03:16, 2471.27it/s]

Extension Namespace 1000128 | module_id | 900000000000012004
Extension Namespace 1000121 | module_id | 900000000000012004
Extension Namespace 1000126 | module_id | 900000000000012004
Merkel cell carcinoma of trunk | module_id | 731000124108
Merkel cell carcinoma of buttock | module_id | 731000124108
Thrombosis of cephalic vein | module_id | 731000124108


652318it [03:17, 2409.75it/s]

Enteropathogenic Escherichia coli serogroup O18 | module_id | 731000124108
Extension Namespace 1000131 | module_id | 900000000000012004
Extension Namespace 1000132 | module_id | 900000000000012004
Extension Namespace 1000133 | module_id | 900000000000012004


652584it [03:17, 2479.85it/s]

In [ ]:
ID_NAMES

In [11]:
! wc -l ../../../data/triplets.tsv

Education about order of protection	type_id	900000000000013009
Education about order of protection	case_significance_id	900000000000448009
Education about order of protection	effective_time	20240301
Education about order of protection	module_id	731000124108
Education about order of protection	groups	groups.3913740_1


In [9]:
! ls ../../../

capacity  data
